## Definition

### Problem Statement  

The goal is to forecast the demand for bikes in dependency of weather conditions like outside temperature and calendric informations e.g. holidays. These information and the demand structure is provided in a set with two years of daily historic data.  
The demand is given as the total daily demand and as a split for registered users and casual users. To increase the quality of the prediction registered user demand and casual user demand will be predicted separately in step two.  
To make predictions machine learning is used to train regressors. Scikit-Learn recommends a support vector regressor (SVR) for this kind of problem and data amount. In addition a deep neuronal network (DNN) regressor is trained for comparison. To find the hyper-parameters for these regressors grid search and randomized search are utilized. Due to the small dataset cross validation is applied.    

> http://scikit-learn.org/stable/tutorial/machine_learning_map/index.html  
> http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR  
> https://github.com/tensorflow/skflow/blob/master/g3doc/api_docs/python/estimators.md  
> http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html  
> http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.RandomizedSearchCV.html

In [2]:
# Import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import calendar

from sklearn.svm import SVR
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from math import sqrt

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Analysis

In [2]:
# Fetching Dataset

bike_data = pd.read_csv("day.csv", header=0)

print("Data read successfully!")

Data read successfully!


In [3]:
bike_data.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


### Data Exploration

In [4]:
# Extracting

feature_cols = bike_data.columns[:-3]  # all columns but last three are features
target_col = bike_data.columns[-1]  # last column is the target

print ("Feature column(s):\n{}\n".format(feature_cols))
print ("Target column:\n{}".format(target_col))

Feature column(s):
Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed'],
      dtype='object')

Target column:
cnt


#### Function to Calculate Profit

In [3]:
def profit(y,y_cap):
    return 3 * np.minimum(y[::1], y_cap[::1]) - 2 * y_cap[::1]
    

### Base Model

#### For the base model the demand for today is the previous days demand. 

In [6]:
y_actual = bike_data[target_col][365:731]  # corresponding targets
y_actual = y_actual.reset_index(drop = True)

In [7]:
y_staged = y_actual.copy()

In [8]:
data = []
data.insert(0, bike_data[target_col][364])
data.insert(0, bike_data[target_col][363])

In [9]:
y_predicted_df = pd.concat([pd.DataFrame(data), y_staged], ignore_index=True)

In [10]:
y_predicted_df.drop(y_predicted_df.tail(2).index,inplace=True)

In [11]:
y_predicted = y_predicted_df[0]


##### Calculate Base Model Profit

In [12]:
print(profit(y_actual,y_predicted).sum())

1442972


### Algorithms and Techniques

#### Demand as Target Variable - cnt

In [4]:
data = pd.read_csv("processed_data_monthly_avg.csv", header=0)
data['instant'] = data['instant'] % 30
X_raw_train = data[0:335]
X_raw_test  = data[335:]

In [5]:
data.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp',
       ...
       'workingday__23', 'workingday__24', 'workingday__25', 'workingday__26',
       'workingday__27', 'workingday__28', 'workingday__29',
       'moving_avg_weekly_cnt', 'moving_avg_monthly_cnt', 'demand_pc_inc'],
      dtype='object', length=338)

In [42]:
cols =[
       "season__1","season__2","season__3","season__4","season__5",
       "weathersit__1","weathersit__2","weathersit__3","weathersit__4","weathersit__5",
        "atemp","hum","windspeed",'cnt__1',
        "mnth","instant","holiday","weekday","workingday",
        "moving_avg_weekly_cnt"]     

In [25]:
##New Set of Columns
cols = [
       'instant', 'season', 'yr', 'mnth', 'holiday', 'weekday','workingday', 
       'atemp__1', 
       'cnt__1', 
       'holiday__1',  
       'hum__1', 
       'temp__1', 
       'weathersit__1',
       'weekday__1',  
       'windspeed__1',  
       'workingday__1', 
       'moving_avg_weekly_cnt'
]

In [43]:
X_train = X_raw_train[cols].values.tolist()
y_train_df = X_raw_train[['cnt']]
y_train = y_train_df['cnt'].tolist()

In [44]:
X_test = X_raw_test[cols].values.tolist()
y_test_df = X_raw_test[['cnt']]
y_test = y_test_df['cnt'].tolist()

In [45]:
data_cnt = data['cnt']

In [46]:
actual_predictions = data_cnt[335:].values

In [47]:
y_for_calculations = data_cnt[357:723].values

### Benchmark

In [48]:
# Training SVR
svr = SVR()
svr.fit(X_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [49]:
# Validation SVR

svr_pred = svr.predict(X_test)
score_svr = r2_score(y_test, svr_pred)
rmse_svr = sqrt(mean_squared_error(y_test, svr_pred))

print("Score SVR: %f" % score_svr)
print("RMSE SVR: %f" % rmse_svr)

Score SVR: -0.934610
RMSE SVR: 2484.461417


In [50]:
#convert demand to integer
svr_pred=np.around(svr_pred)

In [51]:
print(profit(actual_predictions,svr_pred).sum())

1171674.0


## Methodology

### Implementation

The regressors are trained using randomized search and cross-validation to identify the area of the best parameters. Then a grid search is used to tune parameter values of the regressor functions.

> http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html  
> http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.RandomizedSearchCV.html

In [52]:
# Tuning SVR with GridSearch

tuned_parameters = [{'C': [1000, 3000, 10000], 
                     'kernel': ['linear','rbf']}
                   ]

#svr_tuned = GridSearchCV(SVR (C=1), param_grid = tuned_parameters, scoring = 'mean_squared_error') #default 3-fold cross-validation, score method of the estimator
svr_tuned_GS = GridSearchCV(SVR (C=1), param_grid = tuned_parameters, scoring = 'r2', n_jobs=-1) #default 3-fold cross-validation, score method of the estimator

svr_tuned_GS.fit(X_train, y_train)

print (svr_tuned_GS)
print ('\n' "Best parameter from grid search: " + str(svr_tuned_GS.best_params_) +'\n')

GridSearchCV(cv=None, error_score='raise',
       estimator=SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid=[{'C': [1000, 3000, 10000], 'kernel': ['linear', 'rbf']}],
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=0)

Best parameter from grid search: {'C': 10000, 'kernel': 'rbf'}



In [53]:
# Validation - SVR tuned 

svr_tuned_pred_GS = svr_tuned_GS.predict(X_test)

score_svr_tuned_GS = r2_score(y_test, svr_tuned_pred_GS)
rmse_svr_tuned_GS = sqrt(mean_squared_error(y_test, svr_tuned_pred_GS))

print("SVR Results\n")

print("Score SVR: %f" % score_svr)
print("Score SVR tuned GS: %f" % score_svr_tuned_GS)

print("\nRMSE SVR: %f" % rmse_svr)
print("RMSE SVR tuned GS: %f" % rmse_svr_tuned_GS)

SVR Results

Score SVR: -0.934610
Score SVR tuned GS: -1.252493

RMSE SVR: 2484.461417
RMSE SVR tuned GS: 2680.818105


In [54]:
svr_tuned_pred_GS = np.around(svr_tuned_pred_GS)

##Profit Calculation for cnt approach
print(profit(actual_predictions,svr_tuned_pred_GS).sum())

#Profit is just 1.08million!!

1123170.0


In [55]:
# SVR tuned with RandomizesSearch
# may take a while!

# Parameters
param_dist = {  'C': sp_uniform (1000, 10000), 
                'kernel': ['rbf']
             }

n_iter_search = 1

# MSE optimized
#SVR_tuned_RS = RandomizedSearchCV(SVR (C=1), param_distributions = param_dist, scoring = 'mean_squared_error', n_iter=n_iter_search)

# R^2 optimized
SVR_tuned_RS = RandomizedSearchCV(SVR (C=1), param_distributions = param_dist, scoring = 'r2', n_iter=n_iter_search)

# Fit
SVR_tuned_RS.fit(X_train, y_train)

# Best score and corresponding parameters.
print('best CV score from grid search: {0:f}'.format(SVR_tuned_RS.best_score_))
print('corresponding parameters: {}'.format(SVR_tuned_RS.best_params_))

# Predict and score
predict = SVR_tuned_RS.predict(X_test)

score_svr_tuned_RS = r2_score(y_test, predict)
rmse_svr_tuned_RS = sqrt(mean_squared_error(y_test, predict))

best CV score from grid search: -1.528262
corresponding parameters: {'C': 7396.351912361147, 'kernel': 'rbf'}


In [56]:
print('SVR Results\n')

print("Score SVR: %f" % score_svr)
print("Score SVR tuned GS: %f" % score_svr_tuned_GS)
print("Score SVR tuned RS: %f" % score_svr_tuned_RS)

print("\nRMSE SVR: %f" % rmse_svr)
print("RMSE SVR tuned GS: %f" % rmse_svr_tuned_GS)
print("RMSE SVR tuned RS: %f" % rmse_svr_tuned_RS)

SVR Results

Score SVR: -0.934610
Score SVR tuned GS: -1.252493
Score SVR tuned RS: -1.252493

RMSE SVR: 2484.461417
RMSE SVR tuned GS: 2680.818105
RMSE SVR tuned RS: 2680.818105


In [57]:
##Profit Calculation for cnt approach
predict = np.around(predict)
print(profit(actual_predictions,predict).sum())

1123170.0


The tuning works for the SVR.

### DNN Regressor

In [58]:
from sklearn.neural_network import MLPRegressor

In [59]:
import logging
from concurrent.futures import ThreadPoolExecutor, wait
from time import time
from typing import List

In [60]:
bike_model = MLPRegressor(hidden_layer_sizes=(5,),
                                       activation='relu',
                                       solver='adam',
                                       learning_rate='adaptive',
                                       max_iter=1000,
                                       learning_rate_init=0.01,
                                       alpha=0.01)

In [61]:
start_time = int(time() * 1000)
bike_model.fit(X_train, y_train)
end_time = int(time() * 1000)
logging.debug('Finished training universal model')
logging.debug('Training took {} ms'.format(end_time - start_time)) 

In [62]:
predict = bike_model.predict(X_test)

In [63]:
predict = np.around(predict)

In [64]:
print(profit(actual_predictions,predict).sum())
#Profit is 1.52 million!

1404788.0


#### BOOSTING

In [65]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

In [66]:
#evaluation metrics
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error # for regression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score  # for classification
 

In [67]:
models=[RandomForestRegressor(),AdaBoostRegressor(),BaggingRegressor(),SVR(),KNeighborsRegressor()]
model_names=['RandomForestRegressor','GradientBoostingRegressor','AdaBoostRegressor','BaggingRegressor','SVR','KNeighborsRegressor']
rmsle=[]
d={}
for model in range (len(models)):
    clf=models[model]
    print(model_names[model])
    clf.fit(X_train,y_train)
    test_pred=clf.predict(X_test)
    test_pred = np.around(test_pred)
    print(profit(actual_predictions,test_pred).sum())

RandomForestRegressor
1378016.0
GradientBoostingRegressor
1340816.0
AdaBoostRegressor
1324388.0
BaggingRegressor
1171674.0
SVR
1337927.0


In [68]:
#NOW LET'S Dig deeper into each of these ...
#for random forest regresion.
no_of_test=[500]
params_dict={'n_estimators':no_of_test,'n_jobs':[-1],'max_features':["auto",'sqrt','log2'],'max_depth':[10,20,30]}
clf_rf=GridSearchCV(estimator=RandomForestRegressor(),param_grid=params_dict,scoring='neg_mean_squared_error')
clf_rf.fit(X_train,y_train)
pred=clf_rf.predict(X_test)
#model_predictions = convertToPrediction(y_for_calculations,pred)
pred=np.around(pred)
print(profit(actual_predictions,pred).sum())

1317369.0


In [69]:
print("Best params: ", clf_rf.best_params_)

Best params:  {'max_depth': 30, 'max_features': 'sqrt', 'n_estimators': 500, 'n_jobs': -1}


In [70]:

from sklearn.ensemble import RandomForestRegressor
from sklearn import pipeline,metrics,grid_search

regressor = RandomForestRegressor(random_state = 0, max_depth = 40, n_estimators = 500, max_features = 'log2')
estimator = pipeline.Pipeline(steps = [       
    ('model_fitting', regressor)
    ]
)
estimator.fit(X_train, y_train)
test_pred = estimator.predict(X_test)
test_pred = np.around(test_pred)
print(profit(actual_predictions,test_pred).sum())

1319910.0


In [71]:
test_pred

array([1954., 1749., 1886., 1815., 1980., 2122., 2132., 2436., 2601.,
       2506., 2333., 2656., 2240., 2648., 2200., 2146., 1963., 1984.,
       1976., 2050., 1899., 1938., 1737., 1838., 1987., 2251., 2399.,
       2793., 2432., 2736., 2442., 3372., 2758., 2796., 2656., 2631.,
       2410., 2492., 2544., 2716., 2441., 2542., 2617., 2299., 1993.,
       2253., 2318., 2322., 2363., 2694., 2632., 2358., 2430., 3018.,
       2831., 2638., 2478., 2405., 2518., 2636., 3467., 2479., 3096.,
       2405., 2731., 2490., 2443., 3307., 2743., 2907., 2952., 3575.,
       3541., 3625., 3628., 3003., 3492., 3371., 3531., 3466., 3351.,
       3364., 3776., 3550., 3207., 3092., 3490., 4274., 4238., 3473.,
       3465., 3755., 3898., 4481., 4524., 3907., 3674., 3960., 4544.,
       4065., 4024., 3656., 3723., 3953., 4700., 4622., 4655., 4662.,
       4093., 4412., 4415., 4510., 3097., 3292., 3238., 4013., 4169.,
       3814., 3768., 4116., 4259., 4537., 4348., 4400., 4531., 4662.,
       4543., 4439.,

In [45]:
### Gradient Boosting Regressor

In [72]:
#for Gradient Boosting regresion.
no_of_estimators=[100,200,300,400,500]
params_dict={'n_estimators':no_of_estimators,'learning_rate':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
             'max_features':["auto",'sqrt','log2'],'max_depth':[10,20,30,40,50]}
clf_gbr=GridSearchCV(estimator=GradientBoostingRegressor(),param_grid=params_dict,scoring='neg_mean_squared_error')
clf_gbr.fit(X_train,y_train)
pred=clf_gbr.predict(X_test)
pred=np.around(pred)
print(profit(actual_predictions,pred).sum())

1324356.0


In [73]:
print("Best params: ", clf_gbr.best_params_)

Best params:  {'learning_rate': 0.1, 'max_depth': 40, 'max_features': 'sqrt', 'n_estimators': 100}


In [83]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(n_estimators=300, learning_rate=0.1, max_depth = 20)

estimator = pipeline.Pipeline(steps = [       
    ('model_fitting', gbr)
    ]
)
estimator.fit(X_train, y_train)
test_pred = estimator.predict(X_test)
test_pred = np.around(test_pred)
print(profit(actual_predictions,test_pred).sum())

1306172.0


In [49]:
### Bagging Regressor